In [1]:
import sys
sys.path.insert(0, '../')

In [13]:
import rdflib
from wlkernel import *
import random
from itertools import chain
from pathlib import Path
from pprint import pprint
from collections import Counter

In [14]:
rdf_graph = rdflib.Graph().parse('../tests/resources/aifbfixed_complete.n3', format='n3')

In [15]:
affiliation_uri = rdflib.URIRef('http://swrc.ontoware.org/ontology#affiliation')
all_instances = [(t[0], t[2]) for t in rdf_graph.triples( (None, affiliation_uri, None) )]
assert len(all_instances) == len(set(all_instances))

In [41]:
n = 15
random.seed(42)
instances = random.sample(all_instances, n)
instances_id = [x[0] for x in instances]
instances_class = [x[1] for x in instances]
# print('Amount of the values of the classes:\n\t')
# pprint(Counter(instances_class).most_common())

In [17]:
member_uri = rdflib.term.URIRef('http://swrc.ontoware.org/ontology#member')
rdf_graph.remove( (None, member_uri, None) )
rdf_graph.remove( (None, affiliation_uri, None) )

In [18]:
instances_id = [str(x) for x in instances_id]
instances_class = [str(x) for x in instances_class]

In [20]:
kernel_matrix = compute_kernel_matrix(rdf_graph, instances_id, 2, 1)

In [21]:
kernel_matrix_par = compute_kernel_matrix_par(rdf_graph, instances_id, 2, 1)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of  10 | elapsed:    3.6s remaining:    8.5s
[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:    6.3s remaining:    6.3s
[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:    6.5s remaining:    2.8s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    8.0s finished


In [22]:
kernel_matrix_par

[[0, 111.0, 130.0, 79.0, 65.0],
 [111.0, 0, 4526.0, 1438.0, 110.0],
 [130.0, 4526.0, 0, 2421.0, 129.0],
 [79.0, 1438.0, 2421.0, 0, 79.0],
 [65.0, 110.0, 129.0, 79.0, 0]]

In [23]:
kernel_matrix == kernel_matrix_par

True

In [24]:
X = kernel_matrix
y = instances_class

In [25]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [34]:
from sklearn.ensemble import RandomForestClassifier

In [35]:
tree_clf = RandomForestClassifier(n_estimators=75)
tree_clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=75,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [36]:
y_pred = tree_clf.predict(X_test)

In [37]:
def accuracy(y_true, y_pred): return sum([x == y for x, y in zip(y_true, y_pred)]) / len(y_true)

In [38]:
accuracy(y_test, y_pred)

0.0